In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import MultinomialNB

After reading in the dataset, the 'Complete' column needs to be moved so that the train_test_split function is easier to deal with

In [2]:
enron = pd.read_pickle("./enron_cleaned_stops.pkl")
tags = pd.read_pickle("./pos_tags_stops.pkl")

In [3]:
enron.columns

Index(['Message-ID', 'Date', 'From', 'To', 'Subject', 'X-From', 'X-To', 'X-cc',
       'X-bcc', 'X-Folder', 'X-Origin', 'X-FileName', 'content', 'user',
       'Cat_1_level_1', 'Cat_1_level_2', 'Cat_1_weight', 'Cat_2_level_1',
       'Cat_2_level_2', 'Cat_2_weight', 'Cat_3_level_1', 'Cat_3_level_2',
       'Cat_3_weight', 'Cat_4_level_1', 'Cat_4_level_2', 'Cat_4_weight',
       'Cat_5_level_1', 'Cat_5_level_2', 'Cat_5_weight', 'Cat_6_level_1',
       'Cat_6_level_2', 'Cat_6_weight', 'Cat_7_level_1', 'Cat_7_level_2',
       'Cat_7_weight', 'Cat_8_level_1', 'Cat_8_level_2', 'Cat_8_weight',
       'Cat_9_level_1', 'Cat_9_level_2', 'Cat_9_weight', 'Cat_10_level_1',
       'Cat_10_level_2', 'Cat_10_weight', 'Cat_11_level_1', 'Cat_11_level_2',
       'Cat_11_weight', 'Cat_12_level_1', 'Cat_12_level_2', 'Cat_12_weight',
       'labeled', 'Complete'],
      dtype='object')

In [4]:
column_complete = enron.pop('Complete')
enron.insert(0, 'Complete', column_complete)
enron.head()

,Complete,Message-ID,Date,From,To,Subject,X-From,X-To,X-cc,X-bcc,...,Cat_10_level_1,Cat_10_level_2,Cat_10_weight,Cat_11_level_1,Cat_11_level_2,Cat_11_weight,Cat_12_level_1,Cat_12_level_2,Cat_12_weight,labeled
0,re confidential employee informationlenhart i ...,<9831685.1075855725804.JavaMail.evans@thyme>,2001-03-15 14:45:00,frozenset({'phillip.allen@enron.com'}),frozenset({'todd.burke@enron.com'}),Re: Confidential Employee Information/Lenhart,Phillip K Allen,Todd Burke,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True
1,re personal and confidential compensation info...,<21041312.1075855725847.JavaMail.evans@thyme>,2001-03-15 14:11:00,frozenset({'phillip.allen@enron.com'}),frozenset({'kim.bolton@enron.com'}),RE: PERSONAL AND CONFIDENTIAL COMPENSATION INF...,Phillip K Allen,Kim Bolton,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True
2,fw western wholesale activities gas & power co...,<5907100.1075858639941.JavaMail.evans@thyme>,2001-06-20 17:04:51,frozenset({'k..allen@enron.com'}),"frozenset({'matt.smith@enron.com', 'matthew.le...",FW: Western Wholesale Activities - Gas & Power...,"Allen, Phillip K. </O=ENRON/OU=NA/CN=RECIPIENT...","Lenhart, Matthew </O=ENRON/OU=NA/CN=RECIPIENTS...",NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True
3,fw western wholesale activities gas & power co...,<26625142.1075858639964.JavaMail.evans@thyme>,2001-06-20 17:09:00,frozenset({'k..allen@enron.com'}),"frozenset({'matt.smith@enron.com', 'matthew.le...",FW: Western Wholesale Activities - Gas & Power...,"Allen, Phillip K. </O=ENRON/OU=NA/CN=RECIPIENT...","Lenhart, Matthew </O=ENRON/OU=NA/CN=RECIPIENTS...",NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True
4,fw western wholesale activities gas & power co...,<19730598.1075858642129.JavaMail.evans@thyme>,2001-08-09 12:30:58,frozenset({'k..allen@enron.com'}),"frozenset({'matt.smith@enron.com', 'm..tholt@e...",FW: Western Wholesale Activities - Gas & Power...,"Allen, Phillip K. </O=ENRON/OU=NA/CN=RECIPIENT...","Smith, Matt </O=ENRON/OU=NA/CN=RECIPIENTS/CN=M...",NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True


# Split the dataset into a train and test set
X = first 14 columns of the dataframe --> up to the user column

y = the label columns. The remaining columns are all labels, except for the "labeled" column which is True for every sample.

_For now, let's only look at the Cat level 2 column as our topic label. 
This column represents 8 different coarse genres:_
* _1 Company Business, Strategy, etc. (855 cnt.)_
* _2 Purely Personal (49 cnt.)_
* _3 Personal but in professional context (e.g., it was good working with you) (165 cnt.)_
* _4 Logistic Arrangements (meeting scheduling, technical support, etc) (533 cnt.)_
* _5 Employment arrangements (job seeking, hiring, recommendations, etc) (96 cnt.)_
* _6 Document editing/checking (collaboration) (176 cnt.)_
* _7 Empty message (due to missing attachment) (25 cnt.)_
* _8 Empty message (26 cnt.)_


In [5]:
train_X, test_X, train_y, test_y = train_test_split(enron.iloc[:, 0:15], enron.iloc[:, 16], test_size=0.20)

# Supervised Models

## CountVectorizer and Tfidf Vectorizer

In [6]:
cvec = CountVectorizer(analyzer='word',
                      ngram_range=(1,1),
                      max_features=None,
                      stop_words='english',
                      min_df=2,
                      max_df=0.95)

train_counts = cvec.fit_transform(train_X.Complete)

In [7]:
test_counts = cvec.transform(test_X.Complete)

In [8]:
tfvec = TfidfVectorizer(ngram_range=(1, 1),
                       max_features=None,
                       stop_words='english',
                       min_df=2,
                       max_df=0.95)

train_tf = tfvec.fit_transform(train_X['Complete'].fillna(''))

In [9]:
test_tf = tfvec.transform(test_X['Complete'].fillna(' '))

## Naive Bayes Classifier
Sklearn's documentation states that a count vectorizer should be more appropriate for this model. I will try both.

First, count vectorizer

In [10]:
count_n_bayes = MultinomialNB()
count_n_bayes.fit(train_counts, train_y)

count_n_bayes.score(test_counts, test_y)

0.5454545454545454

And now Tfidf

In [11]:
tf_n_bayes = MultinomialNB()
tf_n_bayes.fit(train_tf, train_y)

tf_n_bayes.score(test_tf, test_y)

0.5894428152492669

The tfidf seems to work a little bit better (Depending on the train test split, the tfidf outperforms by up to 4%). This makes sense, because the tfidf adds a weight factor for every word, and this is expected to be important to classification of text. It is interesting that the tfidf matrix works just as well, if not better, than the count matrix despite the documentation recommending the use of a count vector over tfidf.

While the accuracy itself was not great, I am expecting better results from the version that does not include stopwords. Accuracy is also likely affected by the small sample size for both train and test sets.